# support-queue-assistant-rag

In [2]:
# Uncomment and execute this once to download minsearch
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-11 00:36:02--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-11 00:36:02 (43.9 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
# Import libraries
import pandas as pd
import random
import json
from tqdm.auto import tqdm
import minsearch
from openai import OpenAI

## Ingestion

In [5]:
df = pd.read_csv('../data/data-unclean.csv', sep=",")
df.head()

,id,question,category,subcategory,intent,product_feature,response,related_articles,tags
0,0,How do I reset my account password?,Account,Password Reset,Password Recovery,Authentication & Security,To reset your password: 1. Go to the login pag...,/account-security/password-reset;/account-mana...,"password,reset,account,security,login"
1,1,Can I upgrade my subscription plan?,Billing,Subscription Management,Plan Upgrade,Billing & Subscriptions,"Yes, you can upgrade anytime. To do so: 1. Log...",/billing/changing-subscription-plans;/billing/...,"upgrade,subscription,billing,plan,pricing"
2,2,How do I integrate your API with my CRM?,Technical,API Integration,Integration Setup,API & Integrations,To integrate: 1. Generate an API key 2. Review...,/api-reference;/integrations/crm-setup-guide,"API,integration,CRM,developer,technical"
3,3,What's included in your enterprise SLA?,General Inquiry,Service Level Agreement,Contract Information,Enterprise Support,"Our Enterprise SLA includes: 99.99% uptime, 24...",/enterprise/sla;/enterprise/support-tiers,"SLA,enterprise,uptime,support,contract"
4,4,How do I enable two-factor authentication?,Account,Security Settings,Enable 2FA,Authentication & Security,To enable 2FA: 1. Go to 'Account Settings' 2. ...,/account-security/two-factor-authentication;/a...,"2FA,authentication,security,account,login"


In [6]:
df.columns

Index(['id', 'question', 'category', 'subcategory', 'intent',
       'product_feature', 'response', 'related_articles', 'tags'],
      dtype='object')

In [7]:
documents = df.to_dict(orient='records')

In [8]:
documents[0]

{'id': 0,
 'question': 'How do I reset my account password?',
 'category': 'Account',
 'subcategory': 'Password Reset',
 'intent': 'Password Recovery',
 'product_feature': 'Authentication & Security',
 'response': "To reset your password: 1. Go to the login page 2. Click 'Forgot Password' 3. Enter your email 4. Check your email for a reset link 5. Click the link and enter a new password",
 'related_articles': '/account-security/password-reset;/account-management/updating-account-info',
 'tags': 'password,reset,account,security,login'}

In [9]:
index = minsearch.Index(
    text_fields=['question', 'category', 'subcategory', 'intent',
       'product_feature', 'response', 'related_articles', 'tags'],
    keyword_fields=['id']
)

In [10]:
index.fit(documents)

In [11]:
query = 'How do I integrate your API with my CRM?'

In [12]:
index.search(query, num_results=10)

[{'id': 2,
  'question': 'How do I integrate your API with my CRM?',
  'category': 'Technical',
  'subcategory': 'API Integration',
  'intent': 'Integration Setup',
  'product_feature': 'API & Integrations',
  'response': 'To integrate: 1. Generate an API key 2. Review our API docs 3. Use our SDKs or make REST calls 4. Configure your CRM 5. Test the integration thoroughly',
  'related_articles': '/api-reference;/integrations/crm-setup-guide',
  'tags': 'API,integration,CRM,developer,technical'},
 {'id': 14,
  'question': 'How do I generate an API key?',
  'category': 'Technical',
  'subcategory': 'API Management',
  'intent': 'API Key Generation',
  'product_feature': 'API & Integrations',
  'response': "To generate an API key: 1. Go to 'Developer Settings' 2. Click 'API Keys' 3. Select 'Generate New Key' 4. Set permissions for the key 5. Copy and securely store the key (it won't be shown again)",
  'related_articles': '/api-reference/authentication;/api-management/key-security',
  'ta

## RAG Flow

In [13]:
# client = OpenAI(api_key = 'YOUR_KEY')

In [14]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [15]:
prompt_template = """
You're a B2B SaaS company's customer support analyst. Answer the QUESTION based on the CONTEXT from our support queue knowledge database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
question: {question}
category: {category}
subcategory: {subcategory}
intent: {intent}
product_feature: {product_feature}
response: {response}
related_articles: {related_articles}
tags: {tags}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [18]:
question = 'How can I reset my password?'
answer = rag(question)
print(answer)

To reset your password, follow these steps:

1. Go to the login page.
2. Click 'Forgot Password'.
3. Enter your email.
4. Check your email for a reset link.
5. Click the link and enter a new password.


In [19]:
question = 'How can I setup enterprise account?'
answer = rag(question)
print(answer)

To set up an enterprise account, you need to follow these steps:

1. Upgrade to the Enterprise plan.
2. Go to 'Security Settings.'
3. Enable Single Sign-On (SSO).
4. Choose your Identity Provider (IdP) such as Okta or Azure AD.
5. Configure SAML or OAuth.
6. Test the SSO connection.

For more details, you can refer to the related articles on SSO setup and best practices.


## Retrieval evaluation

Make sure that ground truth data was generated using `evaluation-data-generation.ipynb`